In [0]:
%pip install spark-nlp


In [0]:
%pip install transformers

In [0]:
dbutils.library.restartPython()
# Install required libraries
%pip install torch accelerate

# Restart Python kernel
dbutils.library.restartPython()

In [0]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from transformers import pipeline

In [0]:
from pyspark.sql.functions import pandas_udf, col
from pyspark.sql.types import StringType, FloatType, StructType, StructField
import pandas as pd
==
# Initialize the sentiment pipeline (runs once per executor)
@pandas_udf(StringType())
def get_sentiment(texts: pd.Series) -> pd.Series:
    # Initialize pipeline inside UDF (executed on each worker)
    sentiment_analyzer = pipeline(
        "sentiment-analysis",
        model="distilbert-base-uncased-finetuned-sst-2-english",
        device=-1,  # CPU (-1), use 0 for GPU
        truncation=True,
        max_length=512
    )
    
    # Process batch
    results = sentiment_analyzer(texts.tolist(), batch_size=8)
    
    # Return labels
    return pd.Series([result['label'] for result in results])

In [0]:
df = spark.table("workspace.default.silverreview1").limit(1000)

result = df.withColumn("sentiment", get_sentiment(col("text")))


In [0]:
result.select("text", "sentiment").show(truncate=False)

In [0]:
# Extract label and score
result = result.select(
    "review_id",
    "text",
    "stars",
    col("sentiment_data.label").alias("sentiment"),
    col("sentiment_data.score").alias("confidence")
)

result.show(truncate=False)

In [0]:
from transformers import pipeline
import pandas as pd

# Load sentiment pipeline
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

# Get sample texts
texts = [row.text for row in spark.table("workspace.default.silverreview1").limit(100).select("text").collect()]
texts = [t for t in texts if t and len(t) <= 512]

# Run sentiment analysis
results = sentiment_analyzer(texts, batch_size=8)

# Create DataFrame
results_df = pd.DataFrame({
    'Text': [text[:100] + '...' if len(text) > 100 else text for text in texts],  # Truncate long text
    'Sentiment': [result['label'] for result in results],
    'Confidence': [round(result['score'], 3) for result in results]
})

# Display table
print(results_df.to_string(index=False))

# Or use display() in Databricks for interactive table
display(results_df)